Test 1 - Using VGG with last embedded layer and Random Forest at the end

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [2]:
import tensorflow as tf
import numpy as np

In [3]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [4]:
tf.__version__

'2.10.1'

In [5]:
# Analyze + process data
#Filter by number of entries
train_dir = "C:/Users/malik/Desktop/ASL-Aly/ASL-words-translator/IMAGE_vision/train_data"

saved_classes = []

for folder in os.listdir(train_dir):
    if len(os.listdir(os.path.join(train_dir, folder))) >= 15:
        saved_classes.append(folder)
        
    # for file in os.walk(os.path.join(train_dir, dirnames)):
    #     print(file)
(saved_classes)

['afraid',
 'afternoon',
 'again',
 'all',
 'apple',
 'aunt',
 'bad',
 'bathroom',
 'beautiful',
 'black',
 'blue',
 'book',
 'born',
 'boy',
 'boyfriend',
 'bread',
 'brother',
 'brown',
 'bus',
 'bye',
 'cat',
 'cheese',
 'chicken',
 'children',
 'church',
 'class',
 'clothes',
 'coffee',
 'college',
 'color',
 'computer',
 'cook',
 'cousin',
 'cup',
 'dance',
 'daughter',
 'day',
 'deaf',
 'die',
 'doctor',
 'dog',
 'door',
 'draw',
 'drink',
 'eat',
 'egg',
 'english',
 'family',
 'father',
 'fine',
 'finish',
 'fish',
 'forget',
 'france',
 'friend',
 'friendly',
 'girl',
 'gold',
 'good',
 'grandfather',
 'grandmother',
 'green',
 'hair',
 'happy',
 'hat',
 'have',
 'hearing',
 'hello',
 'help',
 'here',
 'home',
 'horse',
 'how',
 'how_many',
 'hungry',
 'hurt',
 'husband',
 'jacket',
 'kitchen',
 'know',
 'late',
 'learn',
 'light',
 'like',
 'live',
 'lost',
 'love',
 'lunch',
 'mad',
 'man',
 'marry',
 'me',
 'meet',
 'milk',
 'morning',
 'mother',
 'my',
 'name',
 'nephew',


In [6]:
# Filter by frames
new_saved = []
for label in saved_classes:
    dir = os.path.join(train_dir, label)
    for file in os.listdir(dir):
        arr = np.load(os.path.join(train_dir, label, file))
        if len(arr) < 2:
            break
        elif os.listdir(dir)[-1] == file:
            new_saved.append(label)

print(len(new_saved))

24


In [106]:
# Data with at least 15 entries, each having at least 2 frames
new_saved = ['cheese',
 'chicken',
 'church',
 'coffee',
 'college',
 'cook',
 'daughter',
 'dog',
 'door',
 'friendly',
 'grandmother',
 'home',
 'hungry',
 'milk',
 'not know',
 'shoes',
 'thursday',
 'time',
 'tomorrow',
 'uncle',
 'vacation',
 'woman',
 'your']

In [107]:
# Add each video into one collective Numpy Array, containing frames (3D objects)
# Train data
train_dir = "C:/Users/malik/Desktop/ASL-Aly/ASL-words-translator/IMAGE_vision/train_data_restricted_labels"
X_train = np.array([])
y_train = np.array([])
for label in new_saved:
    print(label)
    dir = os.path.join(train_dir, label)
    for file in os.listdir(dir):
        arr = np.load(os.path.join(train_dir, label, file))
        for frame in arr:
            if len(X_train) == 0:
                X_train = np.array([frame])
                y_train = np.array([new_saved.index(label)])
            else:
                X_train = np.append(X_train, np.array([frame]), axis=0)
                y_train = np.append(y_train, np.array([new_saved.index(label)]), axis=0)

print(X_train.shape, y_train.shape)

cheese
chicken
church
coffee
college
cook
daughter
dog
door
friendly
grandmother
home
hungry
milk
not know
shoes
thursday
time
tomorrow
uncle
vacation
woman
your
(13912, 256, 256, 3) (13912,)


In [108]:
X_train.shape, y_train.shape

((13912, 256, 256, 3), (13912,))

In [109]:
np.save("C:/Users/malik/Desktop/ASL-Aly/ASL-words-translator/IMAGE_vision/X_train", np.asarray(X_train, dtype=np.float32), allow_pickle=True, fix_imports=True)
np.save("C:/Users/malik/Desktop/ASL-Aly/ASL-words-translator/IMAGE_vision/y_train", np.asarray(y_train, dtype=np.float32), allow_pickle=True, fix_imports=True)

apple
cheese
chicken
church
coffee
college
cook
daughter
dog
door
friendly
grandmother
home
hungry
milk
not know
shoes
thursday
time
tomorrow
uncle
vacation
woman
your
(1404, 256, 256, 3) (1404,)


In [110]:
new_saved = [
 'cheese',
 'chicken',
 'church',
 'coffee',
 'college',
 'cook',
 'daughter',
 'dog',
 'door',
 'friendly',
 'grandmother',
 'home',
 'hungry',
 'milk',
 'not know',
 'shoes',
 'thursday',
 'time',
 'tomorrow',
 'uncle',
 'vacation',
 'woman',
 'your']

In [111]:
X_train = np.load("C:/Users/malik/Desktop/ASL-Aly/ASL-words-translator/IMAGE_vision/X_train.npy")
y_train = np.load("C:/Users/malik/Desktop/ASL-Aly/ASL-words-translator/IMAGE_vision/y_train.npy")
X_train.shape, y_train.shape

((13912, 256, 256, 3), (13912,))

In [116]:
# Function for processing data
def process_data(X_val,y_val, X_test, y_test, directory, new_saved):
    val = False
    for label in new_saved:
        print(label)
        dir = os.path.join(directory, label)
        for file in os.listdir(dir):
            val = not(val)
            print(val)
            arr = np.load(os.path.join(directory, label, file))
            if val:
                for frame in arr:
                    if len(X_val) == 0:
                        X_val = np.array([frame])
                        y_val = np.array([new_saved.index(label)])
                    else:
                        X_val = np.append(X_val, np.array([frame]), axis=0)
                        y_val = np.append(y_val, np.array([new_saved.index(label)]), axis=0)
            else:
                for frame in arr:
                    if len(X_test) == 0:
                        X_test = np.array([frame])
                        y_test = np.array([new_saved.index(label)])
                    else:
                        X_test = np.append(X_test, np.array([frame]), axis=0)
                        y_test = np.append(y_test, np.array([new_saved.index(label)]), axis=0)

    print(X_val.shape, y_val.shape, X_test.shape, y_test.shape)
    return (X_val, y_val, X_test, y_test)

In [117]:
# Val set
X_val = np.array([])
y_val = np.array([])
X_test = np.array([])
y_test = np.array([])
val_dir = "C:/Users/malik/Desktop/ASL-Aly/ASL-words-translator/IMAGE_vision/val_data_restricted_labels"
X_val, y_val, X_test, y_test = process_data(X_val,y_val, X_test, y_test, val_dir, new_saved)

cheese
True
False
True
chicken
False
True
church
False
True
False
True
coffee
False
True
False
True
False
True
college
False
True
False
True
cook
False
True
False
True
daughter
False
True
False
dog
True
False
True
False
True
door
False
True
False
True
friendly
False
True
False
grandmother
True
False
True
False
home
True
False
True
hungry
False
True
False
True
False
milk
True
False
True
False
True
False
True
not know
False
True
False
True
False
shoes
True
False
True
False
True
False
True
False
True
thursday
False
True
time
False
True
tomorrow
False
True
False
uncle
True
False
vacation
True
False
woman
True
False
True
False
your
True
False
True
False
(1307, 256, 256, 3) (1307,) (1318, 256, 256, 3) (1318,)


In [118]:
np.save("C:/Users/malik/Desktop/ASL-Aly/ASL-words-translator/IMAGE_vision/X_val", np.asarray(X_val, dtype=np.float32), allow_pickle=True, fix_imports=True)
np.save("C:/Users/malik/Desktop/ASL-Aly/ASL-words-translator/IMAGE_vision/y_val", np.asarray(y_val, dtype=np.float32), allow_pickle=True, fix_imports=True)
np.save("C:/Users/malik/Desktop/ASL-Aly/ASL-words-translator/IMAGE_vision/X_test", np.asarray(X_test, dtype=np.float32), allow_pickle=True, fix_imports=True)
np.save("C:/Users/malik/Desktop/ASL-Aly/ASL-words-translator/IMAGE_vision/y_test", np.asarray(y_test, dtype=np.float32), allow_pickle=True, fix_imports=True)

In [119]:
X_val = np.load("C:/Users/malik/Desktop/ASL-Aly/ASL-words-translator/IMAGE_vision/X_val.npy")
y_val = np.load("C:/Users/malik/Desktop/ASL-Aly/ASL-words-translator/IMAGE_vision/y_val.npy")
X_test = np.load("C:/Users/malik/Desktop/ASL-Aly/ASL-words-translator/IMAGE_vision/X_test.npy")
y_test = np.load("C:/Users/malik/Desktop/ASL-Aly/ASL-words-translator/IMAGE_vision/y_test.npy")
X_val.shape, y_val.shape, X_test.shape, y_test.shape

((1307, 256, 256, 3), (1307,), (1318, 256, 256, 3), (1318,))

In [67]:
import sklearn
from sklearn.model_selection import train_test_split
X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.5, random_state=42)


In [120]:
#shuffle train data
from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train, random_state=42)
X_train, y_train

(array([[[[ 0.64705884,  0.67058825,  0.70980394],
          [ 0.64705884,  0.67058825,  0.70980394],
          [ 0.64705884,  0.67058825,  0.70980394],
          ...,
          [-0.21568628, -0.09803922,  0.06666667],
          [-0.2       , -0.07450981,  0.07450981],
          [-0.23137255, -0.10588235,  0.04313726]],
 
         [[ 0.6627451 ,  0.6862745 ,  0.7254902 ],
          [ 0.654902  ,  0.6784314 ,  0.7176471 ],
          [ 0.64705884,  0.67058825,  0.70980394],
          ...,
          [-0.20784314, -0.09019608,  0.07450981],
          [-0.1764706 , -0.05098039,  0.09803922],
          [-0.20784314, -0.08235294,  0.06666667]],
 
         [[ 0.62352943,  0.6862745 ,  0.7176471 ],
          [ 0.654902  ,  0.6784314 ,  0.7176471 ],
          [ 0.6392157 ,  0.6627451 ,  0.7019608 ],
          ...,
          [-0.2       , -0.07450981,  0.07450981],
          [-0.18431373, -0.05882353,  0.09019608],
          [-0.20784314, -0.08235294,  0.06666667]],
 
         ...,
 
         [[-

In [121]:
from sklearn.utils import shuffle
X_val, y_val = shuffle(X_val, y_val, random_state=42)

In [122]:
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16

vgg_model = VGG16(weights='imagenet', input_shape=(256, 256, 3), include_top=False)
vgg_model.trainable = False
vgg_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 256, 256, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 256, 256, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 128, 128, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 128, 128, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 128, 128, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 64, 64, 128)       0     

In [123]:
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model

x = vgg_model.get_layer('block5_pool').output
x = Flatten(name='flatten')(x)
x = Dense(4096, activation='relu')(x)
x = Dense(400, activation='relu')(x)
output = Dense(24, activation='softmax')(x)

vgg_model = Model(vgg_model.input, output)

In [124]:
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    "C:/Users/malik/Desktop/ASL-Aly/ASL-words-translator/training_models/weights.{epoch:02d}-{val_accuracy:.2f}",
    monitor='val_accuracy',
    verbose=0,
    save_best_only=True,
    save_weights_only=False,
    mode='auto',
    save_freq='epoch',
)

In [125]:
vgg_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
vgg_model.summary()

Model: "model_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 256, 256, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 256, 256, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 128, 128, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 128, 128, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 128, 128, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 64, 64, 128)       0  

In [126]:
vgg_model.fit(X_train, y_train, epochs=30, batch_size=10, validation_data=(X_val, y_val), callbacks=[checkpoint])

Epoch 1/30
1392/1392 [==============================] - ETA: 0s - loss: 1.4438 - accuracy: 0.6314

INFO:tensorflow:Assets written to: C:/Users/malik/Desktop/ASL-Aly/ASL-words-translator/training_models\weights.01-0.10\assets


INFO:tensorflow:Assets written to: C:/Users/malik/Desktop/ASL-Aly/ASL-words-translator/training_models\weights.01-0.10\assets


1392/1392 [==============================] - 1812s 1s/step - loss: 1.4438 - accuracy: 0.6314 - val_loss: 6.1127 - val_accuracy: 0.0979
Epoch 2/30
1392/1392 [==============================] - 1799s 1s/step - loss: 0.3759 - accuracy: 0.8816 - val_loss: 6.9198 - val_accuracy: 0.0780
Epoch 3/30
1392/1392 [==============================] - 1815s 1s/step - loss: 0.2583 - accuracy: 0.9199 - val_loss: 8.7623 - val_accuracy: 0.0268
Epoch 4/30
 113/1392 [=>............................] - ETA: 27:23 - loss: 0.2456 - accuracy: 0.9168

In [ ]:
from keras import backend as K
import tensorflow as tf

get_fc1_output = K.function([vgg_model.layers[0].input],[vgg_model.layers[-2].output])

test_sample = tf.random.uniform(shape=(1,224,224,3), minval=0, maxval=1)
# test_sample.shape
layer_output = get_fc1_output([test_sample])[0]
layer_output

In [14]:
best_model = tf.keras.models.load_model('C:/Users/malik/Desktop/ASL-Aly/ASL-words-translator/training_models/weights.03-2.05')
best_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 256, 256, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 256, 256, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 128, 128, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 128, 128, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 128, 128, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 64, 64, 128)       0     

In [15]:
best_model.evaluate(X_test, y_test)

9/9 [==============================] - 21s 2s/step - loss: 2.0160 - accuracy: 0.5943


[2.01599383354187, 0.5943060517311096]